# Retrieve Image ClassifAI

In [1]:
from ipyfilechooser import FileChooser
import os
import cv2
import numpy as np
import re
import shutil

# Paths Selection
Once run, you can select files multiple times without rerun the script

### Select Image Path
Select Image Folder

In [2]:
image_path_selector = FileChooser(os.getenv("HOME"))
display(image_path_selector)

FileChooser(path='/home/chuang010896', filename='', title='HTML(value='', layout=Layout(display='none'))', sho…

### Select Label File
Select csv label file

In [3]:
label_file_selector = FileChooser(os.getenv("HOME"))
label_file_selector.filter_pattern = '*.csv'
display(label_file_selector)

FileChooser(path='/home/chuang010896', filename='', title='HTML(value='', layout=Layout(display='none'))', sho…

### Select Output Path
Select Output Folder

*Note: Image will be written in {output_path}/{image_path name} directory*

*Example: Output path = {/home/x/Output}, Image path = /home/x/{Image} => Image will be written in /home/x/Output/Image/xx.jpg*

In [4]:
output_path_selector = FileChooser(os.getenv("HOME"))
output_path_selector.show_only_dirs = True
display(output_path_selector)

FileChooser(path='/home/chuang010896', filename='', title='HTML(value='', layout=Layout(display='none'))', sho…

# Run after selecting all paths

In [ ]:
#Create a Label class to take value from csv file
class Label:
    def __init__(self, x1, y1, x2, y2, label):
        self.x1 = x1
        self.x2 = x2
        self.y1 = y1
        self.y2 = y2
        self.label = label
        
    def width(self):
        return self.x2 - self.x1
    
    def height(self):
        return self.y2 - self.y1
        
        
#Using dictionary to save all bounding boxes and labels for respective images
label_dict = dict()
label_dict.clear()

with open(label_file_selector.selected, mode = 'r', newline='') as file:
    
    #remove csv header
    next(file)
    
    #dictionary loading
    for row in file.readlines():
        
        m = re.search('(.+)(.)(jpg|png|jpeg|JPG|JPEG|PNG|bmp)', row)
        if m:
            found = m.group(0)
        
        filename = found
        
        splitted = row[:-1][len(found)+1:].split(',')
        x1 = int(float(splitted[0]))
        y1 = int(float(splitted[1]))
        x2 = int(float(splitted[2]))
        y2 = int(float(splitted[3]))
        
        #Join back the commas in the label
        label = ','.join(splitted[4:])
        if filename not in label_dict:
            label_dict[filename] = []
        label_dict[filename].append(Label(x1, y1, x2, y2, label))

##Write to Output
#check if image path is a directory
image_path = image_path_selector.selected
image_folder = image_path.split(os.path.sep)[-2]
output_path = os.path.join(output_path_selector.selected,image_folder)

if not os.path.isdir(image_path):
    
    print("not a directory")
else:
    #Create Output Path
    if  not os.path.exists(output_path):
        os.mkdir(output_path)
    
    for filename in os.listdir(image_path):
        
        if filename in label_dict:
            mat = cv2.imread(os.path.join(image_path,filename))
            
            mat = cv2.hconcat([mat, mat])
            
            #Choose color for text & rectangle
            color = (0, 255, 0)
            
            for label in label_dict[filename]:
                
                if label.y2 > mat.shape[0] or label.x2 > mat.shape[1]:
                    continue
                
                #Rectangle size 
                rect_width = label.width()
                rect_height = label.height()
                
                rect_thick = rect_height/100
                font = min(rect_thick * 2.2, 2)
                font_thick = int(2 * font)
                
                cv2.rectangle(mat, (int(label.x1), int(label.y1)), (int(label.x2), int(label.y2)) , color, int(rect_thick))
                cv2.rectangle(mat, (int(label.x1) + mat.shape[1] // 2, int(label.y1)), (int(label.x2) + mat.shape[1] // 2, int(label.y2)) , color, int(rect_thick))
                
                 
                #Text box
                font = cv2.FONT_HERSHEY_COMPLEX_SMALL
                font_color = (255, 255, 255)
                thick = 1
                font_size = 0.9
                (text_width, text_height) = cv2.getTextSize(label.label, font, font_size, thick)[0]
                text_height += 15
                
                mask = np.zeros((text_height, text_width,3), dtype=np.uint8)
                
                mask = cv2.putText(mask, label.label, (0,15), font, font_size, font_color, thick, cv2.LINE_AA)
                
                if rect_height / rect_width > 3:
                    mask = cv2.rotate(mask, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE) 
                
                mask = cv2.resize(mask, (rect_width, rect_height))
                
                mat[label.y1: label.y1 + rect_height, label.x1: label.x1 + rect_width, :] = mask
                
            cv2.imwrite(os.path.join(output_path,filename), mat)